In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('dataset/kidney_stone_data.csv')

In [48]:
df

,treatment,stone_size,success
0,B,large,1
1,A,large,1
2,A,large,0
3,A,large,1
4,A,large,1
...,...,...,...
695,B,small,0
696,B,small,1
697,B,small,1
698,A,large,1


In [49]:
df.isnull().sum()

treatment     0
stone_size    0
success       0
dtype: int64

In [50]:
x = pd.DataFrame(df.iloc[:,[0,1]])

In [51]:
x

,treatment,stone_size
0,B,large
1,A,large
2,A,large
3,A,large
4,A,large
...,...,...
695,B,small
696,B,small
697,B,small
698,A,large


In [52]:
y = df.iloc[:,-1]

In [53]:
y

0      1
1      1
2      0
3      1
4      1
      ..
695    0
696    1
697    1
698    1
699    1
Name: success, Length: 700, dtype: int64

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state =42)

In [56]:
x_train

,treatment,stone_size
82,B,small
51,B,small
220,B,large
669,A,large
545,A,small
...,...,...
71,B,large
106,B,small
270,A,small
435,B,small


In [57]:
x_test['stone_size'].unique()

array(['large', 'small'], dtype=object)

In [58]:
y_train

82     1
51     0
220    0
669    1
545    1
      ..
71     1
106    1
270    1
435    0
102    1
Name: success, Length: 560, dtype: int64

In [59]:
y_test

158    1
500    1
396    1
155    0
321    0
      ..
24     1
218    1
431    1
281    0
110    1
Name: success, Length: 140, dtype: int64

In [60]:
from sklearn.preprocessing import OneHotEncoder

In [61]:
from sklearn.preprocessing import OrdinalEncoder

In [62]:
from sklearn.compose import ColumnTransformer

In [63]:
trfn = ColumnTransformer(transformers=[
    ('a',OneHotEncoder(drop = 'first',sparse = False,handle_unknown='ignore'),[0])
],remainder = 'passthrough')

In [64]:
trfn1 = ColumnTransformer(transformers = [
    ('t2',OrdinalEncoder(categories = [['large', 'small']],unknown_value = -1,handle_unknown='use_encoded_value'),[1])
],remainder = "passthrough")

In [72]:
from sklearn.feature_selection import SelectKBest,chi2
trfn2 = SelectKBest(score_func = chi2,k=2)

In [73]:
from sklearn.linear_model import LogisticRegression

In [74]:
trfn3 = LogisticRegression()

In [75]:
from sklearn.pipeline import Pipeline

In [76]:
pipe = Pipeline([('trfn',trfn),
                 ('trfn1',trfn1),
                 ('trfn2',trfn2),
                 ('trfn3',trfn3)])

In [80]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trfn',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('a',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [0])])),
                ('trfn1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t2',
                                                  OrdinalEncoder(categories=[['large',
                                                                              'small']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [1])])),
                ('trfn2',
                 SelectKBest(k=2,
                             score_func=<function chi2 at 0x000001AE41AD7EE0>)),
                ('trfn3', LogisticRegression())])

In [78]:
from sklearn import set_config

In [79]:
set_config(display = 'diagram')

In [81]:
y_pred = pipe.predict(x_test)

In [87]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [83]:
from sklearn.metrics import accuracy_score

In [85]:
accuracy_score(y_test,y_pred)

0.7571428571428571

In [90]:
from sklearn.model_selection import cross_val_score

In [95]:
cross_val_score(pipe,x_train,y_train,cv = 5,scoring = 'accuracy').mean()

0.8142857142857143

In [96]:
import pickle

In [97]:
pickle.dump(pipe,open('ksd.pkl','wb'))